# TorchMD API tutorial

## System setup

We use the `moleculekit` library for reading the input topologies and starting coordinates

In [2]:
from moleculekit.molecule import Molecule
import os

testdir = "../test-data/prod_alanine_dipeptide_amber/"
mol = Molecule(os.path.join(testdir, "structure.prmtop"))  # Reading the system topology
mol.read(os.path.join(testdir, "input.coor"))  # Reading the initial simulation coordinates
mol.read(os.path.join(testdir, "input.xsc"))  # Reading the box dimensions

Next we will load a forcefield file and use the above topology to extract the relevant parameters which will be used for the simulation

In [5]:
from ..paddlemd.forcefields.forcefield import ForceField
from paddlemd.parameters import Parameters
import torch

precision = torch.float
device = "cuda:0"

ff = ForceField.create(mol, os.path.join(testdir, "structure.prmtop"))
parameters = Parameters(ff, mol, precision=precision, device=device)

ValueError: attempted relative import beyond top-level package

Now we can create a `System` object which will contain the state of the system during the simulation, including:
1. The current atom coordinates
1. The current box size
1. The current atom velocities
1. The current atom forces

In [3]:
from paddlemd.integrator import maxwell_boltzmann
from paddlemd.systems import System

system = System(mol.numAtoms, nreplicas=1, precision=precision, device=device)
system.set_positions(mol.coords)
system.set_box(mol.box)
system.set_velocities(maxwell_boltzmann(parameters.masses, T=300, replicas=1))

Lastly we will create a `Force` object which will be used to evaluate the potential on a given `System` state

In [11]:
from paddlemd.forces import Forces
bonded = ["bonds", "angles", "dihedrals", "impropers", "1-4"]
bonded = ["dihedrals"]
# forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5)
forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5, terms=bonded)
# Evaluate current energy and forces. Forces are modified in-place
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=True)

print(Epot)
print(system.forces)

[{'dihedrals': 10.262107849121094, 'external': 0.0}]
tensor([[[-1.1108e-01,  1.6149e-02, -1.0404e+00],
         [ 1.8305e-01, -1.0476e-03,  1.6788e+00],
         [ 5.5393e-03,  5.7595e-02, -2.9841e-02],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]], device='cuda:0')


## Dynamics

For performing the dynamics we will create an `Integrator` object for integrating the time steps of the simulation as well as a `Wrapper` object for wrapping the system coordinates within the periodic cell

In [12]:
from paddlemd.integrator import Integrator
from paddlemd.wrapper import Wrapper

langevin_temperature = 300  # K
langevin_gamma = 0.1
timestep = 1  # fs

integrator = Integrator(system, forces, timestep, device, gamma=langevin_gamma, T=langevin_temperature)
wrapper = Wrapper(mol.numAtoms, mol.bonds if len(mol.bonds) else None, device)

In [16]:
from paddlemd.minimizers import minimize_bfgs

minimize_bfgs(system, forces, steps=500)  # Minimize the system

Iter  Epot            fmax    
   0    20.109737    2.481930
   1    16.807449    1.724286
   2    22.018057    11.455074
   3    16.203218    26.515063
   4    16.007851    6.448757
   5    23.911276    44.388491
   6    15.678329    8.338514
   7    13.330523    119.376837
   8    23.211462    6.625708
   9    19.027088    8.426726
  10    17.093735    9.266688
  11    12.646109    95.903555
  12    11.705179    61.297083
  13    16.962624    6.941132
  14    11.705179    61.297083
  15    21.706810    10.844314
  16    18.531084    26.332549
  17    11.141059    45.311407
  18    21.985168    76.741523
  19    10.931394    43.353227
  20    11.286078    81.720612
  21    10.281906    17.560959
  22    12.046317    135.483850
  23    10.213875    3.632023
  24    10.059407    1.820187
  25    9.990677    2.355876
  26    18.351007    75.369637
  27    9.988084    2.364855
  28    9.964596    2.464586
  29    18.355877    72.311387
  30    9.964068    2.466532
  31    9.959239    2.49

Create a CSV file logger for the simulation which keeps track of the energies and temperature.

In [17]:
from paddlemd.utils import LogWriter

logger = LogWriter(path="logs/", keys=('iter','ns','epot','ekin','etot','T'), name='monitor.csv')

Writing logs to  logs/monitor.csv


Now we can finally perform the full dynamics

In [18]:
from tqdm import tqdm 
import numpy as np

FS2NS = 1E-6 # Femtosecond to nanosecond conversion

steps = 50000 # 1000
output_period = 10
save_period = 100
traj = []

trajectoryout = "mytrajectory.npy"

iterator = tqdm(range(1, int(steps / output_period) + 1))
Epot = forces.compute(system.pos, system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
    currpos = system.pos.detach().cpu().numpy().copy()
    traj.append(currpos)
    
    if (i*output_period) % save_period  == 0:
        np.save(trajectoryout, np.stack(traj, axis=2))

    logger.write_row({'iter':i*output_period,'ns':FS2NS*i*output_period*timestep,'epot':Epot,'ekin':Ekin,'etot':Epot+Ekin,'T':T})

100%|██████████| 5000/5000 [08:38<00:00,  4.95it/s]
